# 1. Import package

In [11]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
import pickle
import csv

# 2. Save login cookie

In [ ]:
options = webdriver.ChromeOptions()
# options.add_argument('proxy-server=106.122.8.54:3128')
options.add_argument(r'--user-data-dir=C:\Users\ADMIN\AppData\Local\Google\Chrome\User Data\Default')

driver = uc.Chrome(
    options=options,
    driver_executable_path='D:/chromedriver'
)
url = 'https://twitter.com/login'
driver.get(url)
sleep(3)
username = driver.find_element(By.XPATH,"//input[@name='text']")
username.send_keys('Claire011098')
next_button = driver.find_element(By.XPATH,"//span[contains(text(),'Next')]")
next_button.click()
sleep(3)
# email = driver.find_element(By.XPATH,"//input[@name='text']")
# email.send_keys('nguyenhaiyen1632@gmail.com')
# next_button = driver.find_element(By.XPATH,"//span[contains(text(),'Next')]")
# next_button.click()
# sleep(3)
password = driver.find_element(By.XPATH,"//input[@name='password']")
password.send_keys('Hvqy011098@')
log_in = driver.find_element(By.XPATH,"//span[contains(text(),'Log in')]")
log_in.click()
sleep(10)

pickle.dump(driver.get_cookies(), open("my_cookie.pkl","wb"))

# 3. Load the login cookie to reuse the saved cookie

In [2]:
options = webdriver.ChromeOptions()
options.add_argument(r'--user-data-dir=C:\Users\ADMIN\AppData\Local\Google\Chrome\User Data\Default')

driver = uc.Chrome(
    options=options,
    driver_executable_path='D:/chromedriver'
)
url = 'https://twitter.com/login'
driver.get(url)
cookies = pickle.load(open("my_cookie.pkl","rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://twitter.com/login")

# 4. Search by hashtag

In [3]:
hashtag = "#MachineLearning"
search_box = driver.find_element(By.XPATH,"//input[@data-testid='SearchBox_Search_Input']")
search_box.send_keys(hashtag)
search_box.send_keys(Keys.ENTER)
sleep(3)
latest_results = driver.find_element(By.XPATH,"//span[contains(text(),'Top')]")
latest_results.click()

# 5. Scraping data

In [8]:
# Khởi tạo các mảng lưu trữ dữ liệu khi thu thập
tweets_texts = []
likes = []
retweets = []
replies = []
views = []
bookmarks = []
usernames = []
names = []
postdates = []
item_target_count = 30 # Khởi tạo điều kiện dừng vòng lặp (số mẫu dữ liệu muốn thu thập)

actions = ActionChains(driver)
while True:
    sleep(5) # Thời gian dừng để load nội dung
    actions.send_keys(Keys.END).perform() # Điều khiển cuộn xuống trang để thu thập thêm các tweet
    sleep(10)
    articles = driver.find_elements(By.XPATH,"//article[@data-testid='tweet']")
    for article in articles:
        try:
            display_name = article.find_element(By.XPATH,'.//span').text
            # display_name = article.find_element(By.XPATH,'//*[@class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l r-1awozwy r-18u37iz"]/div[1]/div/a/div/div[1]/span/span').text
        except NoSuchElementException:
            display_name = ''
        names.append(display_name)
        username = article.find_element(By.XPATH,'.//span[contains(text(), "@")]').text
        usernames.append(username)
        postdate = article.find_element(By.XPATH,'.//time').get_attribute('datetime')
        postdates.append(postdate)

        tweets = article.find_elements(By.CSS_SELECTOR, 'div[data-testid=tweetText]')
        stats = article.find_elements(By.CSS_SELECTOR, 'div[role=group]')
        for idx, stat in enumerate(stats):
            # Thêm nội dung của tweet vào danh sách tweets_texts
            tweet_text = tweets[idx].text
            tweets_texts.append(tweet_text)

            # Tách các thống kê (số lượng comment, repost, like, view, bookmark) từ thuộc tính aria-label
            stat_labels = stat.get_attribute('aria-label').split(',')
            print(stat_labels)

        # Khởi tạo các giá trị mặc định cho các thống kê
        like_count = '0'
        retweet_count = '0'
        reply_count = '0'
        view_count = '0'
        bookmark_count = '0'
        
        # Lặp qua các thống kê để phân loại chúng
        for label in stat_labels:
            parts = label.split()
            count = parts[0]  # Số lượng
            label_type = parts[1]  # Loại thống kê (comment, repost, like, view, bookmark)

            if "reply" in label_type or "replies" in label_type:
                reply_count = count
            elif "repost" in label_type:
                retweet_count = count
            elif "like" in label_type:
                like_count = count
            elif "bookmark" in label_type:
                bookmark_count = count
            elif "view" in label_type:
                view_count = count
        
        # Thêm các giá trị vào danh sách tương ứng
        likes.append(like_count)
        retweets.append(retweet_count)
        replies.append(reply_count)
        views.append(view_count)
        bookmarks.append(bookmark_count)

    # Xóa các tweet trùng lặp và kiểm tra xem đã thu thập đủ các tweet duy nhất chưa
    Tweets2 = list(set(tweets_texts))
    print(len(Tweets2))
    if len(Tweets2) > item_target_count:
        break


['5 reposts', ' 11 likes', ' 1 bookmark', ' 3438 views']
['9 replies', ' 56 reposts', ' 411 likes', ' 155 bookmarks', ' 24019 views']
['2 replies', ' 11 reposts', ' 14 likes', ' 6 bookmarks', ' 1373 views']
['2 replies', ' 8 reposts', ' 15 likes', ' 1 bookmark', ' 1323 views']
['3 replies', ' 4 reposts', ' 15 likes', ' 5 bookmarks', ' 902 views']
['2 replies', ' 15 reposts', ' 85 likes', ' 34 bookmarks', ' 9355 views']
['1 repost', ' 7 likes', ' 316 views']
7
['3 replies', ' 4 reposts', ' 15 likes', ' 5 bookmarks', ' 902 views']
['2 replies', ' 15 reposts', ' 85 likes', ' 34 bookmarks', ' 9355 views']
['1 repost', ' 7 likes', ' 316 views']
['7 replies', ' 31 reposts', ' 48 likes', ' 6 bookmarks', ' 2559 views']
['538 replies', ' 10264 reposts', ' 30135 likes', ' 976 bookmarks']
['8 replies', ' 77 reposts', ' 175 likes']
['1 reply', ' 5 reposts', ' 12 likes', ' 3 bookmarks', ' 694 views']
['120 replies', ' 3829 reposts', ' 4348 likes', ' 4 bookmarks']
12
['3 replies', ' 4 reposts', ' 15

In [7]:
driver.refresh() #Refresh the page if needed

# 6. Save data to csv file

In [9]:
from datetime import datetime
time = datetime.now().strftime("%d%m%Y_%H%M%S")
data = {
    'Username':usernames,
    'Name': names,
    'Tweet': tweets_texts,
    'TimeStamp': postdates,
    'Comment': replies,
    'ReTweet': retweets,
    'Like': likes,
    'View': views,
    'Bookmark': bookmarks
}

df = pd.DataFrame(data)
df.to_csv(f'tweets_data_{time}.csv', index=False,encoding='utf-8')